In [2]:
from tensorflow.keras.utils import img_to_array
import imutils
from keras.models import load_model
import numpy as np
import cv2

In [4]:
detection_model_path = '/content/drive/MyDrive/haarcascade_frontalface_default.xml'
emotion_model_path = '/content/drive/MyDrive/emotion_detection_model.h5'
face_detection = cv2.CascadeClassifier(detection_model_path)
emotion_classifier = load_model(emotion_model_path, compile=False)
EMOTIONS = ["disgust","happy","sad","neutral"]

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
def get_results(emotion_word):
  NUM_RECOMMEND=10
  if emotion_word =='happy':
      return (data[(data['kmeans'] == 2) & (data['popularity'] > 75) & (data['song_name'] != 'NaN')][['song_name','artists']].sample(NUM_RECOMMEND))
  elif emotion_word =='sad':
      return (data[(data['kmeans'] == 1) & (data['popularity'] > 66) & (data['song_name'] != 'NaN')][['song_name','artists']].sample(NUM_RECOMMEND))
  elif emotion_word=='disgust':
      return (data[(data['kmeans'] == 0) & (data['popularity'] > 75) & (data['song_name'] != 'NaN')][['song_name','artists']].sample(NUM_RECOMMEND))
  elif emotion_word=='neutral':
      return (data[(data['kmeans'] == 3) & (data['popularity'] > 69) & (data['song_name'] != 'NaN')][['song_name','artists']].sample(NUM_RECOMMEND))

In [8]:
from keras.preprocessing import image
def emotion_testing():
        cap=cv2.VideoCapture(0)
        while True:
            ret,test_img=cap.read()# captures frame and returns boolean value and captured image
            if not ret:
                continue
            gray_img= cv2.cvtColor(test_img, cv2.COLOR_BGR2GRAY)

            faces_detected = face_detection.detectMultiScale(gray_img, 1.32, 5)


            for (x,y,w,h) in faces_detected:
                cv2.rectangle(test_img,(x,y),(x+w,y+h),(255,0,0),thickness=7)
                roi_gray=gray_img[y:y+w,x:x+h]#cropping region of interest i.e. face area from  image
                roi_gray=cv2.resize(roi_gray,(48,48))
                img_pixels = img_to_array(roi_gray)
                img_pixels = np.expand_dims(img_pixels, axis = 0)
                img_pixels /= 255

                predictions = emotion_classifier.predict(img_pixels)
                print(predictions)

                 #find max indexed array
                max_index = np.argmax(predictions[0])
                print(max_index)
                predicted_emotion = EMOTIONS[max_index]
                print(predicted_emotion)
                print(get_results(predicted_emotion))

                cv2.putText(test_img, predicted_emotion, (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)

            resized_img = cv2.resize(test_img, (1000, 700))
            cv2.imshow('Facial emotion analysis ',resized_img)

            if cv2.waitKey(0) & 0xFF == ord('q'):
                break
        cap.release()
        cv2.destroyAllWindows


In [ ]:
a = emotion_testing()
a

1/1 [==============================] - 0s 422ms/step
[[1.0018645e-04 8.8174410e-02 2.9459570e-02 8.8226581e-01]]
3
neutral
                            song_name                              artists
16120                    All the Time                     ['Zara Larsson']
87364        I Like The Sound Of That                    ['Rascal Flatts']
78063       2 Hearts (feat. Gia Koka)   ['Sam Feldt', 'Sigma', 'Gia Koka']
97704                    Amor Tumbado                    ['Natanael Cano']
97214  Outside (feat. Ellie Goulding)  ['Calvin Harris', 'Ellie Goulding']
86259     I Hate Everything About You                 ['Three Days Grace']
97778                 i'm so tired...              ['Lauv', 'Troye Sivan']
87464                  Loud and Heavy                       ['Cody Jinks']
70537                         Orphans                         ['Coldplay']
86961                       Marry You                       ['Bruno Mars']
1/1 [==============================] - 0s 100ms/step